- Assignments:

1. Weather model:

For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:
- First, load the dataset from the weatherinszeged table from Thinkful's database.

In [ ]:
# Import some libraries and packages:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import acf
from scipy.stats import bartlett
from scipy.stats import levene
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings(action="ignore")
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format

In [5]:
# Connect to dataset:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from weatherinszeged', con = engine)
engine.dispose()

In [6]:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.47,7.39,0.89,14.12,251.00,15.83,0.00,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.36,7.23,0.86,14.26,259.00,15.83,0.00,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.38,9.38,0.89,3.93,204.00,14.96,0.00,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.29,5.94,0.83,14.10,269.00,15.83,0.00,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.76,6.98,0.83,11.04,259.00,15.83,0.00,1016.51,Partly cloudy throughout the day.


- Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. 
- What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

In [9]:
# Build a regression model:
# Y is the target variable:
Y = (df['apparenttemperature']-df['temperature'])
# X is the feature set which includes humidity, windspeed:
X = df[['humidity', 'windspeed']]
# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sun, 08 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:41:19   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- R-squared is a proportion (between 0 and 1) that expresses how much variance in the outcome variable is explained by the explanatory variables in the model. 
- Adjusted R-squared does the same job as R-squared, but it is adjusted according to the number of features included in the model. Because,R-squared is biased as it tends to increase with the number of explanatory variables.
- So, in this model: R-squared and adjusted R-squared of the model both are 0.288.
- I think it is not satisfactory, because both R-squared and adjusted R-squared have low values around 0.29 and the higher values are better. So, 71% of the variation in the target variable is unexplained by the features or explanatory variables in the model. So, the results are not satisfactory. 
- however in some cases, it is hard to explain target variables also, the best models can explain a very low portion of the information in the target. So, in these cases, satisfaction of the results is a relative success.

- Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?

In [8]:
# Y is the target variable
Y = (df['apparenttemperature']-df['temperature'])

# This is the interaction between humidity and windspeed
df["humidity_windspeed"] = df.humidity * df.windspeed

# X is the feature set
X = df[['humidity', 'windspeed', 'humidity_windspeed']]

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sun, 08 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:24:28   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0.033      2.511      0.012       0.018       0.149
humidity               0.1775      0.043      4.133      0.000       0.093       0.262
windspeed              0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- So, in this model: R-squared and adjusted R-squared of the model both are 0.341.
- By Including the interaction the R-squared and adjusted R-squared values are increased. So, 64% of the variation in the target variable is unexplained by the explanatory variables in the model which is decreased compare to previous one. So, this is an improvement upon the previous model.

- Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

In [10]:
# Build a regression model:
# Y is the target variable:
Y = (df['apparenttemperature']-df['temperature'])
# X is the feature set which includes humidity, windspeed, and visibility:
X = df[['humidity', 'windspeed', 'visibility']]
# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Sun, 08 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:41:40   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- In this model, R-squared is 0.304 and Adjust R-squared is 0.303. Comparing to first model, Both R-squared and adjusted R-squared increased by adding visibility as an explanatory variable in the model and improved the model.
- Comparing this model with interaction term model, this model's improvement is lower. Interaction term can explain more portion of target variable, so it is more useful than visibility to explain the information in the target.

- Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.?

- For both AIC and BIC, the lower the value the better.

In the first model; both AIC and BIC are 340900, In the second model; both AIC and BIC are 333400, and for the third model, both AIC and BIC are 338800. So, the second model with lowest  AIC and BIC is the best of these three models.

2. House prices model:

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

- Load the houseprices data from Thinkful's database.

In [11]:
# Connect to the data base:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

In [13]:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.00,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


- Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC?

In [14]:
# I convert two categorical variables to numbers and concat to dataset:
df = pd.concat([df,pd.get_dummies(df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
df = pd.concat([df,pd.get_dummies(df.utilities, prefix="utilities", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(df.mszoning, prefix="mszoning", drop_first=True).columns) + list(pd.get_dummies(df.utilities, prefix="utilities", drop_first=True).columns)


In [15]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set which includes: threessnporch, mosold and dummy columns:
X = df[['threessnporch','mosold'] + dummy_column_names]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     25.98
Date:                Mon, 09 Dec 2019   Prob (F-statistic):           1.13e-33
Time:                        00:17:59   Log-Likelihood:                -18458.
No. Observations:                1460   AIC:                         3.693e+04
Df Residuals:                    1452   BIC:                         3.697e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             6.348e+04   2.44e+04      2.602      0.009    1.56e+04    1.11e+05
threessnporch       84.8014     67.158      1.263      0.207     -46.936     216.539
mosold            1435.3758    729.741      1.967      0.049       3.916    2866.835
mszoning_FV       1.415e+05   2.55e+04      5.545      0.000    9.15e+04    1.92e+05
mszoning_RH       5.732e+04   3.03e+04      1.894      0.058   -2045.410    1.17e+05
mszoning_RL       1.181e+05   2.39e+04      4.950      0.000    7.13e+04    1.65e+05
mszoning_RM       5.387e+04   2.43e+04      2.217      0.027    6195.085    1.02e+05
utilities_NoSeWa -4.556e+04   7.52e+04     -0.606      0.545   -1.93e+05    1.02e+05
==============================================================================
Omnibus:                      692.221   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5041.452
Skew:                           2.081   Prob(JB):                         0.00
Kurtosis:                      11.096   Cond. No.                     1.13e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In this model:

- F_test is 25.98 and p_value is 0
- R-square and Adjust R-squared both are around 0.11
- AIC is 36930 and BIC is 36970

According to the R-squared Adjust R-squared almost 89% of the variance in the target variable is unexplained by explanatory variable in the model. So, there is a large room for improvement.

- Do you think your model is satisfactory? If so, why?

I think it is not satisfactory, because both R-squared and adjusted R-squared have low values around 0.11 and the higher values are better. So, 89% of the variation in the target variable is unexplained by the features or explanatory variables in the model. So, the results are not satisfactory. 

- In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables?

??? im not sure  i choose the continus variable with low relationship with targget variable is correct or not???
??? also i couldnt understand why sample result use log for y  and 'To this end, we apply log(1+x) transformation of NumPy.' what does mean??? is it possible to add dummies variable like countinuse one ?

- In order to improve the goodness of fit of your model, I add lotarea and overallqual variables and remove threessnporch variable.

In [18]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set which includes: overallqual, lotarea, mosold and dummy columns:
X = df[['overallqual','lotarea', 'mosold'] + dummy_column_names]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.671
Method:                 Least Squares   F-statistic:                     372.5
Date:                Mon, 09 Dec 2019   Prob (F-statistic):               0.00
Time:                        00:54:59   Log-Likelihood:                -17729.
No. Observations:                1460   AIC:                         3.548e+04
Df Residuals:                    1451   BIC:                         3.552e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.047e+05   1.52e+04     -6.893      0.000   -1.35e+05   -7.49e+04
overallqual       4.337e+04    908.926     47.721      0.000    4.16e+04    4.52e+04
lotarea              1.2720      0.123     10.370      0.000       1.031       1.513
mosold            -259.6280    444.393     -0.584      0.559   -1131.350     612.094
mszoning_FV       -379.4045   1.58e+04     -0.024      0.981   -3.14e+04    3.06e+04
mszoning_RH       1120.6259   1.84e+04      0.061      0.952    -3.5e+04    3.73e+04
mszoning_RL       1.419e+04   1.46e+04      0.969      0.333   -1.45e+04    4.29e+04
mszoning_RM      -1.204e+04   1.48e+04     -0.811      0.417   -4.11e+04    1.71e+04
utilities_NoSeWa -5.024e+04   4.57e+04     -1.100      0.271    -1.4e+05    3.93e+04
==============================================================================
Omnibus:                      634.892   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8524.474
Skew:                           1.656   Prob(JB):                         0.00
Kurtosis:                      14.365   Cond. No.                     5.55e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.55e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In this model:

- F_test is 372.5 and p_value is 0 and compare to the first model (F_test is 25.98) is improved. Because the model with higher F statistic is superior to the other one.
- R-square and Adjust R-squared both are around 0.67. So, almost 33% of the variance in the target variable is unexplained by explanatory variable in the model. So, compare to the first model, this model is improved.
- AIC is 35480 and BIC is 35520. Comparing with first model, AIC and BIC both are less then first model. So,this model is better than the first model.

- So according to above, this model has a better goodness of fit than the first model. 